In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

# fmt:off
_ = [sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent / "python"] if str(path) not in sys.path] #fmt:on

In [ ]:
import os

import openai
from langsmith import traceable
from langsmith.wrappers import wrap_openai

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

# Auto-trace LLM calls in-context
client = wrap_openai(openai.Client())


@traceable  # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}], model="gpt-3.5-turbo"
    )
    return result.choices[0].message.content


pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?

In [ ]:
hasattr(my_car, "year")

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors.llmlingua_filter import (
    LLMLinguaCompressor,
)
from langchain_openai import ChatOpenAI

## Or use LLMLingua-2-small model
compressor2= LLMLinguaCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True, 
    lingua=None

)

In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [4]:
from devtools import debug
debug(multiply.args_schema)

/tmp/ipykernel_338564/4160336977.py:2 <module>
    multiply.args_schema: <class 'pydantic.v1.main.multiplySchema'> (ModelMetaclass)


pydantic.v1.main.multiplySchema

In [5]:
debug(pydantic.v1.main.multiplySchema)

NameError: name 'pydantic' is not defined

In [ ]:
debug()

In [6]:
import tiktoken

from typing import List

encoder = tiktoken.encoding_for_model("gpt-4")


def preprocess_func(text: str) -> List[str]:
    # Lowercase the input text
    lowered = text.lower()

    # Convert the lowered text into tokens
    tokens = encoder.encode(lowered)

    # Stringify the tokens 
    return [str(token) for token in tokens]

In [9]:
preprocess_func("Indemnification")

['485', '336', '77', '2461']

In [11]:
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

file = "fraisier vert cultivé fruits moins appréciés fait acidité plus forte espèces fait objet cueillette consommation personnelle"
stemmer = FrenchStemmer()
tokens = word_tokenize(file)
french_stem = [stemmer.stem(word) for word in word_tokenize(file)]
stemmed_text = ' '.join([stemmer.stem(word) for word in word_tokenize(file)])
print(stemmed_text)

fraisi vert cultiv fruit moin appréci fait acid plus fort espec fait objet cueillet consomm personnel


In [ ]:
import spacy
nlp_fr = spacy.load('fr_core_news_sm')

In [33]:

text = "J'ai mangé ainsi des pommes hier HEKK (fsdfsf) deuxième"
stop_words = nlp_fr.Defaults.stop_words
stop_words.update([",", ";", "(", ")", ":", "[", "]"])
tokens = nlp_fr(text)
lemmas = [token.lemma_.lower() for token in nlp_fr(text)]
filtered = [token for token in lemmas if token not in stop_words]
filtered

['manger', 'pomme', 'hier', 'hekk', 'fsdfsf']

In [30]:
stop_words

{'(',
 ')',
 ',',
 ':',
 ';',
 '[',
 ']',
 'a',
 'abord',
 'afin',
 'ah',
 'ai',
 'aie',
 'ainsi',
 'ait',
 'allaient',
 'allons',
 'alors',
 'anterieur',
 'anterieure',
 'anterieures',
 'antérieur',
 'antérieure',
 'antérieures',
 'apres',
 'après',
 'as',
 'assez',
 'attendu',
 'au',
 'aupres',
 'auquel',
 'aura',
 'auraient',
 'aurait',
 'auront',
 'aussi',
 'autre',
 'autrement',
 'autres',
 'autrui',
 'aux',
 'auxquelles',
 'auxquels',
 'avaient',
 'avais',
 'avait',
 'avant',
 'avec',
 'avoir',
 'avons',
 'ayant',
 'bas',
 'basee',
 'bat',
 "c'",
 'car',
 'ce',
 'ceci',
 'cela',
 'celle',
 'celle-ci',
 'celle-la',
 'celle-là',
 'celles',
 'celles-ci',
 'celles-la',
 'celles-là',
 'celui',
 'celui-ci',
 'celui-la',
 'celui-là',
 'cent',
 'cependant',
 'certain',
 'certaine',
 'certaines',
 'certains',
 'certes',
 'ces',
 'cet',
 'cette',
 'ceux',
 'ceux-ci',
 'ceux-là',
 'chacun',
 'chacune',
 'chaque',
 'chez',
 'ci',
 'cinq',
 'cinquantaine',
 'cinquante',
 'cinquantième',
 'cin